In [ ]:
"""
Advanced Time Series Forecasting with Prophet and Hyperparameter Optimization
-----------------------------------------------------------------------------
Single-file, production-ready Python script that:
- programmatically generates a complex daily time series (seasonality, holidays, changepoints, trend shifts, noise)
- implements time-series cross-validation (rolling/expanding window)
- performs systematic hyperparameter search for Prophet (grid or Bayesian stub)
- fits a baseline Holt-Winters Exponential Smoothing model
- compares models using MASE, RMSE, MAE
- saves dataset and results

Usage:
    python prophet_time_series_project.py --generate-data --run-all

Requirements (pip install):
    prophet          # 'prophet' package (formerly fbprophet). Use 'pip install prophet'
    numpy
    pandas
    scikit-learn
    scikit-optimize  # optional, used for Bayesian search if available
    statsmodels
    matplotlib

Notes:
- This script is intentionally modular. Replace or extend the tuning strategy
  (grid_search_prophet -> bayes_search_prophet) as needed.
- For reproducibility, seeds are set where applicable.

"""

import os
import argparse
import warnings
from dataclasses import dataclass
from typing import Dict, Any, List, Tuple

import numpy as np
import pandas as pd
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error, mean_absolute_error
import matplotlib.pyplot as plt

# Try imports for Prophet and statsmodels; give clear error messages if missing.
try:
    from prophet import Prophet
except Exception as e:
    Prophet = None

try:
    from statsmodels.tsa.holtwinters import ExponentialSmoothing
except Exception as e:
    ExponentialSmoothing = None

# ----------------------------- Utility metrics ------------------------------

def mase(y_true: np.ndarray, y_pred: np.ndarray, y_train: np.ndarray) -> float:
    """Mean Absolute Scaled Error (MASE).
    MASE = MAE / MAE_naive, where MAE_naive is mean absolute naive forecast error on training set
    Uses seasonal naive if seasonality known (we'll use naive with lag=1 here for daily data).
    """
    n = y_train.shape[0]
    if n < 2:
        raise ValueError("y_train must have at least 2 observations for MASE")
    naive_errors = np.abs(y_train[1:] - y_train[:-1])
    mae_naive = np.mean(naive_errors)
    mae = np.mean(np.abs(y_true - y_pred))
    return mae / mae_naive if mae_naive != 0 else np.inf

# ----------------------------- Data generation ------------------------------

def generate_synthetic_daily_series(start_date: str = '2018-01-01',
                                    periods: int = 3 * 365,
                                    seed: int = 42) -> pd.DataFrame:
    """Generate a synthetic daily time series with features often seen in real data:
    - long-term trend with changepoints
    - yearly seasonality (sinusoidal)
    - weekly seasonality (weekday effect)
    - special 'holiday' effects for certain dates
    - random noise and occasional outliers

    Returns a DataFrame with columns ['ds','y'] suitable for Prophet and other libraries.
    """
    np.random.seed(seed)
    dates = pd.date_range(start=start_date, periods=periods, freq='D')
    t = np.arange(periods)

    # Base trend: piecewise linear with changepoints
    trend = 0.03 * t  # base slow upward trend
    # introduce changepoints: at 1/3 and 2/3 of series we change slope
    cp1 = int(periods * 0.33)
    cp2 = int(periods * 0.66)
    trend[cp1:] += 0.1 * (t[cp1:] - cp1)
    trend[cp2:] -= 0.08 * (t[cp2:] - cp2)

    # Yearly seasonality: yearly cycle approximated with period 365
    yearly = 10 * np.sin(2 * np.pi * t / 365.25)

    # Weekly seasonality: weekday effects
    weekday = np.array([0.0 if d.weekday() < 5 else 5.0 for d in dates])  # weekend bump

    # Holiday effect: create random holiday dates with spikes
    rng = np.random.default_rng(seed)
    num_holidays = max(6, int(periods / 200))
    holiday_indices = rng.choice(periods, size=num_holidays, replace=False)
    holiday_effect = np.zeros(periods)
    for hi in holiday_indices:
        holiday_effect[max(0, hi - 1):min(periods, hi + 2)] += rng.normal(20, 5)

    # Add noise and occasional outliers
    noise = np.random.normal(0, 2.5, size=periods)
    outliers = np.zeros(periods)
    outlier_positions = rng.choice(periods, size=int(periods * 0.005), replace=False)
    outliers[outlier_positions] = rng.normal(40, 15, size=outlier_positions.shape[0])

    y = 50 + trend + yearly + weekday + holiday_effect + noise + outliers

    df = pd.DataFrame({'ds': dates, 'y': y})
    # Provide a boolean holidays flag for later use
    holidays = pd.DataFrame({
        'ds': dates[holiday_indices],
        'holiday': ['synthetic_holiday'] * len(holiday_indices)
    })

    return df.reset_index(drop=True), holidays

# ---------------------------- Cross-validation ------------------------------

def rolling_origin_splits(df: pd.DataFrame, initial: int, horizon: int, step: int) -> List[Tuple[np.ndarray, np.ndarray]]:
    """Generate rolling origin (expanding window) train/test indices for time series CV.

    Parameters
    ----------
    df : DataFrame with index in time order
    initial : int, size of initial training window
    horizon : int, size of each test fold
    step : int, how many steps to move forward between folds

    Returns
    -------
    List of (train_idx, test_idx) tuples (numpy arrays of indices)
    """
    n = df.shape[0]
    splits = []
    start = initial
    while start + horizon <= n:
        train_idx = np.arange(0, start)
        test_idx = np.arange(start, start + horizon)
        splits.append((train_idx, test_idx))
        start += step
    return splits

# ----------------------------- Prophet tuning --------------------------------

def grid_search_prophet(df: pd.DataFrame,
                        holidays: pd.DataFrame = None,
                        param_grid: Dict[str, List[Any]] = None,
                        cv_initial: int = 365,
                        cv_horizon: int = 90,
                        cv_step: int = 90,
                        max_evals: int = 100) -> Dict[str, Any]:
    """Simple grid search over Prophet hyperparameters using rolling-origin CV.

    This is intentionally implemented as a straightforward loop so it works without
    scikit-optimize or Optuna. If you have those libraries available, replace with
    Bayesian search for speed.

    Returns the best_params and a DataFrame of all results.
    """
    if Prophet is None:
        raise ImportError("Prophet package not found. Install it with 'pip install prophet'")

    # Default grid
    if param_grid is None:
        param_grid = {
            'changepoint_prior_scale': [0.001, 0.01, 0.1, 0.5],
            'seasonality_prior_scale': [1.0, 5.0, 10.0],
            'seasonality_mode': ['additive', 'multiplicative'],
            'weekly_seasonality': [True, False],
            'yearly_seasonality': [True, False]
        }

    # Build list of param combinations
    import itertools
    keys = list(param_grid.keys())
    combos = list(itertools.product(*(param_grid[k] for k in keys)))

    results = []
    splits = rolling_origin_splits(df, initial=cv_initial, horizon=cv_horizon, step=cv_step)

    for combo in combos:
        params = dict(zip(keys, combo))
        fold_metrics = []
        # Perform CV
        for train_idx, test_idx in splits:
            train_df = df.iloc[train_idx].copy()
            test_df = df.iloc[test_idx].copy()

            m = Prophet(
                changepoint_prior_scale=params['changepoint_prior_scale'],
                seasonality_mode=params['seasonality_mode'],
                weekly_seasonality=params['weekly_seasonality'],
                yearly_seasonality=params['yearly_seasonality']
            )
            if 'seasonality_prior_scale' in params:
                # Prophet does not accept seasonality_prior_scale in constructor directly prior to some versions.
                # We'll set it via add_seasonality or tune default priors where applicable.
                m.seasonality_prior_scale = params['seasonality_prior_scale']

            if holidays is not None and not holidays.empty:
                # Prophet expects holidays df with columns 'ds' and 'holiday'
                m.holidays = holidays

            # Fit model
            m.fit(train_df)
            future = test_df[['ds']].copy()
            forecast = m.predict(future)
            y_true = test_df['y'].values
            y_pred = forecast['yhat'].values

            rmse_ = mean_squared_error(y_true, y_pred, squared=False)
            mae_ = mean_absolute_error(y_true, y_pred)
            mase_ = mase(y_true, y_pred, train_df['y'].values)
            fold_metrics.append({'rmse': rmse_, 'mae': mae_, 'mase': mase_})

        # Aggregate folds
        avg_rmse = np.mean([fm['rmse'] for fm in fold_metrics])
        avg_mae = np.mean([fm['mae'] for fm in fold_metrics])
        avg_mase = np.mean([fm['mase'] for fm in fold_metrics])

        result_row = {**params, 'rmse': avg_rmse, 'mae': avg_mae, 'mase': avg_mase}
        results.append(result_row)

    results_df = pd.DataFrame(results)
    # Sort by primary metric (mase) then rmse
    results_df = results_df.sort_values(['mase', 'rmse']).reset_index(drop=True)
    best = results_df.iloc[0].to_dict()
    return {'best_params': {k: best[k] for k in keys}, 'results_df': results_df}

# ----------------------------- Prophet fit & forecast ------------------------

def fit_prophet(df_train: pd.DataFrame, df_future: pd.DataFrame, params: Dict[str, Any], holidays: pd.DataFrame = None) -> pd.DataFrame:
    """Fit Prophet with given params on df_train and forecast for df_future. Returns forecast DataFrame.
    df_future should contain 'ds' column for prediction dates.
    """
    if Prophet is None:
        raise ImportError("Prophet package not found. Install it with 'pip install prophet'")

    m = Prophet(
        changepoint_prior_scale=params.get('changepoint_prior_scale', 0.05),
        seasonality_mode=params.get('seasonality_mode', 'additive'),
        weekly_seasonality=params.get('weekly_seasonality', True),
        yearly_seasonality=params.get('yearly_seasonality', True)
    )
    # assign seasonality prior if available
    if 'seasonality_prior_scale' in params:
        m.seasonality_prior_scale = params['seasonality_prior_scale']

    if holidays is not None and not holidays.empty:
        m.holidays = holidays

    m.fit(df_train)
    forecast = m.predict(df_future)
    return forecast

# ----------------------------- Baseline model -------------------------------

def fit_holt_winters(train: pd.Series, forecast_index: pd.DatetimeIndex, seasonal_periods: int = 7) -> pd.DataFrame:
    """Fit Holt-Winters Exponential Smoothing and forecast.

    Returns a DataFrame with index = forecast_index and a 'yhat' column.
    """
    if ExponentialSmoothing is None:
        raise ImportError("statsmodels package or Holt-Winters not available. Install statsmodels")

    # Use additive trend and seasonal components as a reasonable baseline
    model = ExponentialSmoothing(train, trend='add', seasonal='add', seasonal_periods=seasonal_periods)
    fitted = model.fit(optimized=True)
    yhat = fitted.forecast(len(forecast_index))
    return pd.DataFrame({'ds': forecast_index, 'yhat': yhat.values})

# ----------------------------- Evaluation & plotting -----------------------

def evaluate_forecast(y_true: np.ndarray, y_pred: np.ndarray, y_train: np.ndarray) -> Dict[str, float]:
    rmse_ = mean_squared_error(y_true, y_pred, squared=False)
    mae_ = mean_absolute_error(y_true, y_pred)
    mase_ = mase(y_true, y_pred, y_train)
    return {'rmse': rmse_, 'mae': mae_, 'mase': mase_}


def plot_forecasts(df: pd.DataFrame, prophet_forecast: pd.DataFrame, hw_forecast: pd.DataFrame, title: str = 'Forecast comparison') -> None:
    plt.figure(figsize=(14, 6))
    plt.plot(df['ds'], df['y'], label='observed', linewidth=1)
    # Prophet's yhat aligns by ds
    plt.plot(prophet_forecast['ds'], prophet_forecast['yhat'], label='Prophet yhat')
    plt.plot(hw_forecast['ds'], hw_forecast['yhat'], label='Holt-Winters yhat')
    plt.legend()
    plt.title(title)
    plt.xlabel('ds')
    plt.ylabel('y')
    plt.tight_layout()
    plt.show()

# ----------------------------- Pipeline / Main -----------------------------

@dataclass
class ProjectConfig:
    start_date: str = '2018-01-01'
    periods: int = 3 * 365
    seed: int = 42
    cv_initial: int = 365
    cv_horizon: int = 90
    cv_step: int = 90
    output_dir: str = 'output'


def run_pipeline(config: ProjectConfig, save_outputs: bool = True) -> Dict[str, Any]:
    os.makedirs(config.output_dir, exist_ok=True)

    # 1. Generate data
    df, holidays = generate_synthetic_daily_series(start_date=config.start_date, periods=config.periods, seed=config.seed)
    if save_outputs:
        df.to_csv(os.path.join(config.output_dir, 'simulated_series.csv'), index=False)
        holidays.to_csv(os.path.join(config.output_dir, 'simulated_holidays.csv'), index=False)

    # Split into train/holdout (last 180 days holdout)
    holdout_days = 180
    train_df = df.iloc[:-holdout_days].reset_index(drop=True)
    holdout_df = df.iloc[-holdout_days:].reset_index(drop=True)

    # 2. Prophet hyperparameter tuning (grid search)
    param_grid = {
        'changepoint_prior_scale': [0.001, 0.01, 0.05, 0.1],
        'seasonality_prior_scale': [1.0, 5.0],
        'seasonality_mode': ['additive', 'multiplicative'],
        'weekly_seasonality': [True, False],
        'yearly_seasonality': [True, False]
    }

    tuning = grid_search_prophet(train_df, holidays=holidays, param_grid=param_grid,
                                 cv_initial=config.cv_initial, cv_horizon=config.cv_horizon, cv_step=config.cv_step)

    best_params = tuning['best_params']
    tuning['results_df'].to_csv(os.path.join(config.output_dir, 'prophet_tuning_results.csv'), index=False)

    # 3. Fit optimized Prophet on entire training set and forecast holdout
    prophet_forecast = fit_prophet(train_df, holdout_df[['ds']].copy(), best_params, holidays=holidays)
    prophet_eval = evaluate_forecast(holdout_df['y'].values, prophet_forecast['yhat'].values, train_df['y'].values)

    # 4. Baseline: Holt-Winters
    hw_forecast = fit_holt_winters(train_df['y'], holdout_df['ds'], seasonal_periods=7)
    hw_eval = evaluate_forecast(holdout_df['y'].values, hw_forecast['yhat'].values, train_df['y'].values)

    # 5. Save and plot
    if save_outputs:
        prophet_forecast[['ds', 'yhat']].to_csv(os.path.join(config.output_dir, 'prophet_holdout_forecast.csv'), index=False)
        hw_forecast[['ds', 'yhat']].to_csv(os.path.join(config.output_dir, 'hw_holdout_forecast.csv'), index=False)
        pd.DataFrame([prophet_eval]).to_csv(os.path.join(config.output_dir, 'prophet_holdout_metrics.csv'), index=False)
        pd.DataFrame([hw_eval]).to_csv(os.path.join(config.output_dir, 'hw_holdout_metrics.csv'), index=False)

    # (Optional) Plot — comment out in non-interactive/run environments
    try:
        plot_forecasts(pd.concat([train_df, holdout_df], axis=0).reset_index(drop=True),
                       prophet_forecast[['ds', 'yhat']], hw_forecast[['ds', 'yhat']],
                       title='Observed vs Prophet vs Holt-Winters on holdout')
    except Exception:
        pass

    summary = {
        'best_params': best_params,
        'prophet_eval': prophet_eval,
        'hw_eval': hw_eval,
        'tuning_results_csv': os.path.join(config.output_dir, 'prophet_tuning_results.csv'),
        'prophet_forecast_csv': os.path.join(config.output_dir, 'prophet_holdout_forecast.csv'),
        'hw_forecast_csv': os.path.join(config.output_dir, 'hw_holdout_forecast.csv'),
        'dataset_csv': os.path.join(config.output_dir, 'simulated_series.csv')
    }

    return summary


def main():
    parser = argparse.ArgumentParser(description='Advanced Time Series Forecasting with Prophet')
    parser.add_argument('--generate-data', action='store_true', help='Only generate and save the synthetic dataset')
    parser.add_argument('--run-all', action='store_true', help='Run the full pipeline (tuning + baseline + evaluation)')
    parser.add_argument('--output-dir', type=str, default='output', help='Output directory for CSVs')
    args = parser.parse_args()

    config = ProjectConfig(output_dir=args.output_dir)

    if args.generate_data:
        df, holidays = generate_synthetic_daily_series(start_date=config.start_date, periods=config.periods, seed=config.seed)
        os.makedirs(config.output_dir, exist_ok=True)
        df.to_csv(os.path.join(config.output_dir, 'simulated_series.csv'), index=False)
        holidays.to_csv(os.path.join(config.output_dir, 'simulated_holidays.csv'), index=False)
        print(f"Saved simulated dataset to {os.path.join(config.output_dir, 'simulated_series.csv')}")
        return

    if args.run_all:
        print("Running full pipeline. This may take time depending on Prophet installation and tuning grid size.")
        summary = run_pipeline(config)
        print("Pipeline complete. Summary:")
        for k, v in summary.items():
            print(f"  {k}: {v}")
        return

    parser.print_help()


if __name__ == '__main__':
    main()
